In [ ]:
# Load environment variables
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# Create embeddings and a vector db out of stored index
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Meilisearch

embeddings = OpenAIEmbeddings()
vectordb = Meilisearch(embedding=embeddings, index_name='quipbot-test')

In [ ]:
# Create a retriever from the vector db
retriever = vectordb.as_retriever()

query = "Who was the lord of House Arion?"
retriever.get_relevant_documents(query=query)

In [ ]:
# Create a conversation agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0, max_tokens=500, )
conversational_memory = ConversationBufferWindowMemory(memory_key='chat_history', k=5, return_messages=True)
# conversation = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, chain_type='stuff')
qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")

In [ ]:
# Test the retriever
qa.run(query)

In [ ]:
# Let's convert the retrieval chain into a tool
from langchain.agents import Tool

tools = [
    Tool(
        name='QuipBot Knowledge Base', 
        func=qa.run, 
        description='use this tool when answering general knowledge queries to get more information about the topic'
    )
]

In [ ]:
# Now, we can initialize the agent
from langchain.agents import initialize_agent
from langchain.agents import AgentType

system_message = "You are Elderan, an intelligent AI assistant who will help people to know more about the Kingdom of Eldoria. Answer questions that are asked to you. If you don't have proper and enough information politely say you don't know."

agent = initialize_agent(
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory,
    agent_kwargs={
             "system_message": system_message
            }
)


In [ ]:
agent.run('Who was the lord of House Arion?')

In [ ]:
agent.run('Does he have any son?')

In [ ]:
agent.run('Who is Eamon Snow?')

In [ ]:
agent.run('Is he loyal to his house?')

In [ ]:
agent.run('Who are you?')